In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
codontab = {
    'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',    # serine
    'TTC': 'F', 'TTT': 'F',    # Fenilalanine
    'TTA': 'L', 'TTG': 'L',    # Leucine
    'TAC': 'Y', 'TAT': 'Y',    # Tirosine
    'TAA': '*', 'TAG': '*',    # Stop
    'TGC': 'C', 'TGT': 'C',    # Cisteine
    'TGA': '*',    # Stop
    'TGG': 'W',    # Triptofane
    'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',    # Leucine
    'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',    # Proline
    'CAC': 'H', 'CAT': 'H',    # Histidine
    'CAA': 'Q', 'CAG': 'Q',    # Glutamine
    'CGA': 'R', 'CGC': 'R',    # arginine
    'CGG': 'R', 'CGT': 'R',    # arginine
    'ATA': 'I', 'ATC': 'I', 'ATT': 'I',    # Isoleucine
    'ATG': 'M',    # Methionine
    'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',    # Treonine
    'AAC': 'N', 'AAT': 'N',    # asparagine
    'AAA': 'K', 'AAG': 'K',    # lisine
    'AGC': 'S', 'AGT': 'S',    # serine
    'AGA': 'R', 'AGG': 'R',    # arginine
    'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',    # valine
    'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',    # alanine
    'GAC': 'D', 'GAT': 'D',    # Aspartic Acid
    'GAA': 'E', 'GAG': 'E',    # Glutamic Acid
    'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G'     # glicine
}

In [3]:
# map the DNA variants to the amino acid changes
def map_dna_to_aa(dna_seq):
    aa_seq = []
    for i in range(0, len(dna_seq), 3):
        codon = dna_seq[i:i+3]
        if codon in codontab:
            aa = codontab[codon]
            if aa == '*':
                break
            aa_seq.append(aa)
        else:
            print(f"Unknown codon: {codon}")
            aa_seq.append('X')
    if aa == "*" and i < len(dna_seq) - 3:
        print(f"Stop codon encountered at position {i} in sequence ({i//3} AA in sequence).")
    aa_seq.append('*')
        #print(f"Warning: Sequence is longer than expected. Remaining sequence: {dna_seq[i:]}")
    return ''.join(aa_seq)

In [4]:
def apply_dna_mutations(mutations):
    # Create a list to hold the mutated sequence
    mutated_seq = list(refseq)
    
    # Apply the mutation
    for mut in mutations.split('_'):
        ref_bp, pos, new_bp = mut[0], mut[1:-1], mut[-1]
        if "DEL" in mut:
            mut = mut.replace("DEL", "")
            pos = mut[1:]
            new_bp = None
        pos = int(pos) - 1  # Convert to 0-based index
        assert refseq[pos] == ref_bp, f"Reference base {ref_bp} does not match at position {pos + 1}."
        if new_bp is None:
            # Deletion: remove the base
            mutated_seq.pop(pos)
        else:
            # Substitution: replace the base
            mutated_seq[pos] = new_bp
    
    return ''.join(mutated_seq)

In [5]:
def get_aa_mutations(ref_seq, mut_seq):
    """ List the mutations between two amino acid sequences. """
    # Convert the sequences to lists of amino acids
    ref_aa = list(ref_seq)
    mut_aa = list(mut_seq)
    
    # Find the positions of the mutations
    mutations = []
    for i in range(len(mut_aa)):
        if ref_aa[i] != mut_aa[i]:
            mutations += [f"{ref_aa[i]}{i+1}{mut_aa[i]}"]
            #mutations.append((i+1, ref_aa[i], mut_aa[i]))
    
    return '_'.join(mutations)

In [6]:
df = pd.read_csv("250507/250507_variants.csv")
df

,barcode_plate,name,refseq,variant,index,Plate,Well,Barcode,ID,P value,Mixed Well,Variant,Average mutation frequency,Alignment Count,Average error rate,P adj. value
0,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,0.0,GsAdh_2025-04-16_plate5,A1,RB01_NB01,GsAdh_2025-04-16_plate5_A1,NaN,False,#PARENT#,NaN,7.0,NaN,NaN
1,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,1.0,GsAdh_2025-04-16_plate5,A2,RB01_NB02,GsAdh_2025-04-16_plate5_A2,NaN,False,#PARENT#,NaN,15.0,NaN,NaN
2,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,2.0,GsAdh_2025-04-16_plate5,A3,RB01_NB03,GsAdh_2025-04-16_plate5_A3,NaN,False,#PARENT#,NaN,26.0,NaN,NaN
3,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,3.0,GsAdh_2025-04-16_plate5,A4,RB01_NB04,GsAdh_2025-04-16_plate5_A4,NaN,False,#PARENT#,NaN,109.0,NaN,NaN
4,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,4.0,GsAdh_2025-04-16_plate5,A5,RB01_NB05,GsAdh_2025-04-16_plate5_A5,NaN,False,#PARENT#,NaN,121.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,91.0,GsAdh_2025-04-16_plate6,H8,RB02_NB92,GsAdh_2025-04-16_plate6_H8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,92.0,GsAdh_2025-04-16_plate6,H9,RB02_NB93,GsAdh_2025-04-16_plate6_H9,NaN,False,#PARENT#,NaN,39.0,NaN,NaN
189,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,93.0,GsAdh_2025-04-16_plate6,H10,RB02_NB94,GsAdh_2025-04-16_plate6_H10,NaN,False,#PARENT#,NaN,17.0,NaN,NaN
190,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,94.0,GsAdh_2025-04-16_plate6,H11,RB02_NB95,GsAdh_2025-04-16_plate6_H11,NaN,False,#PARENT#,NaN,11.0,NaN,NaN


In [7]:
refseq = df["refseq"].unique()[0]
refseq

'ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGAAAGAAGTGGAAAAACCTAAGATCTCATACGGGGAAGTATTAGTGCGCATCAAAGCGTGTGGGGTATGCCATACAGACTTGCATGCCGCACATGGCGACTGGCCTGTAAAGCCTAAACTGCCTCTCATTCCTGGCCATGAAGGCGTCGGTGTAATTGAAGAAGTAGGTCCTGGGGTAACACATTTAAAAGTTGGAGATCGCGTAGGTATCCCTTGGCTTTATTCGGCGTGCGGTCATTGTGACTATTGCTTAAGCGGACAAGAAACATTATGCGAACGTCAACAAAACGCTGGCTATTCCGTCGATGGTGGTTATGCTGAATATTGCCGTGCTGCAGCCGATTATGTCGTAAAAATTCCTGATAACTTATCGTTTGAAGAAGCCGCTCCAATCTTTTGCGCTGGTGTAACAACATATAAAGCGCTCAAAGTAACAGGCGCAAAACCAGGTGAATGGGTAGCCATTTACGGTATCGGCGGGCTTGGACATGTCGCAGTCCAATACGCAAAGGCGATGGGGTTAAACGTCGTTGCTGTCGATTTAGGTGATGAAAAACTTGAGCTTGCTAAACAACTTGGTGCAGATCTTGTCGTCAATCCGAAACATGATGATGCAGCACAATGGATAAAAGAAAAAGTGGGCGGTGTGCATGCGACTGTCGTCACAGCTGTTTCAAAAGCCGCGTTCGAATCAGCCTACAAATCCATTCGTCGCGGTGGTGCTTGCGTACTCGTCGGATTACCGCCGGAAGAAATACCTATTCCAATTTTCGATACAGTATTAAATGGAGTAAAAATTATTGGTTCTATCGTTGGTACGCGCAAAGACTTACAAGAGGCACTTCAATTTGCAGCAGAAGGAAAAGTAAAAACAATTGTCGAAGTGCAACCGCTTGAAAACATTAACGACGTATTCGATCGTATGTTAAAAGGGCAAATTAACGGCCGCGTCGTG

In [8]:
aa_refseq = map_dna_to_aa(refseq)

In [10]:
df.columns

Index(['barcode_plate', 'name', 'refseq', 'variant', 'index', 'Plate', 'Well',
       'Barcode', 'ID', 'P value', 'Mixed Well', 'Variant',
       'Average mutation frequency', 'Alignment Count', 'Average error rate',
       'P adj. value'],
      dtype='object')

In [11]:
df_mut = df[df["P value"] < 0.05]
print(len(df_mut))

95


In [12]:
df_mut.Variant.value_counts()

Variant
A240G_C348T_T442A_T717C_A747C             3
T174A_A531G_A600C_A647G                   3
G421A_T612C_A695T_A972T                   3
T390A                                     2
T85DEL_G146A_C322A_A746G_T875C_A890DEL    2
                                         ..
T612C                                     1
C280T_T842A_T852G                         1
T534C                                     1
A423G_A512T_A600T_G727A                   1
G538A_A783G                               1
Name: count, Length: 79, dtype: int64

In [13]:
df_mut["AA_seq"] = df_mut.Variant.apply(lambda x: map_dna_to_aa(apply_dna_mutations(x)))

Stop codon encountered at position 150 in sequence (50 AA in sequence).
Stop codon encountered at position 360 in sequence (120 AA in sequence).
Stop codon encountered at position 150 in sequence (50 AA in sequence).
Stop codon encountered at position 423 in sequence (141 AA in sequence).
Stop codon encountered at position 744 in sequence (248 AA in sequence).
Stop codon encountered at position 999 in sequence (333 AA in sequence).
Stop codon encountered at position 360 in sequence (120 AA in sequence).
Stop codon encountered at position 273 in sequence (91 AA in sequence).
Stop codon encountered at position 387 in sequence (129 AA in sequence).
Stop codon encountered at position 387 in sequence (129 AA in sequence).
Stop codon encountered at position 339 in sequence (113 AA in sequence).
Stop codon encountered at position 144 in sequence (48 AA in sequence).
Stop codon encountered at position 84 in sequence (28 AA in sequence).
Stop codon encountered at position 471 in sequence (157 A

/var/folders/49/v33xr4hd5_g2qr31s1v5jqp4j7ccgq/T/ipykernel_16149/3658698572.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mut["AA_seq"] = df_mut.Variant.apply(lambda x: map_dna_to_aa(apply_dna_mutations(x)))


In [14]:
df_mut["AA_mutations"] = df_mut.AA_seq.apply(lambda x: get_aa_mutations(aa_refseq, x))

/var/folders/49/v33xr4hd5_g2qr31s1v5jqp4j7ccgq/T/ipykernel_16149/737037107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mut["AA_mutations"] = df_mut.AA_seq.apply(lambda x: get_aa_mutations(aa_refseq, x))


In [15]:
df_mut.to_csv("250507/250507_variants_aa_mut.csv", index=False)

In [16]:
df_mut.head(4)

,barcode_plate,name,refseq,variant,index,Plate,Well,Barcode,ID,P value,Mixed Well,Variant,Average mutation frequency,Alignment Count,Average error rate,P adj. value,AA_seq,AA_mutations
5,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,5.0,GsAdh_2025-04-16_plate5,A6,RB01_NB06,GsAdh_2025-04-16_plate5_A6,0.000000e+00,False,C148T_G251A_A648C,0.988095,84.0,0.000000,NaN,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,P50S_G84D_K216N
8,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,8.0,GsAdh_2025-04-16_plate5,A9,RB01_NB09,GsAdh_2025-04-16_plate5_A9,5.481139e-141,False,T439C,0.971429,70.0,0.014286,5.261893e-139,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,F147L
9,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,9.0,GsAdh_2025-04-16_plate5,A10,RB01_NB10,GsAdh_2025-04-16_plate5_A10,6.028883e-34,False,T209C_T752C,0.916667,12.0,0.083333,5.787728e-32,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,V70A_I251T
10,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,10.0,GsAdh_2025-04-16_plate5,A11,RB01_NB11,GsAdh_2025-04-16_plate5_A11,9.406419e-13,False,A941G,1.000000,6.0,0.000000,9.030162e-11,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,E314G


## Now map the plate and wells

In [30]:
# load the stats file previously generated
stats_file = "250507/20250423_GsAdh_epPCR_Batch1_CF15_R2_linear_fits_22000s.xlsx"

plate_data = pd.DataFrame()
for plate in range(1,7):
    sheet_name = f"Plate {plate} - Sheet1"
    sheet_data = pd.read_excel(stats_file, 
                       sheet_name=sheet_name, 
                       header=0, 
                       index_col=0).reset_index(drop=False, names="Well")
    sheet_data["Plate"] = plate
    plate_data = pd.concat([plate_data, sheet_data])

plate_data

,Well,slope,r2,y-intercept,1/slope,Plate
0,C04,8.666805e-07,0.939892,0.236645,1.153828e+06,1
1,B05,7.662667e-07,0.839006,0.255263,1.305029e+06,1
2,F03,7.279289e-07,0.932977,0.231986,1.373760e+06,1
3,B07,7.000757e-07,0.902764,0.266747,1.428417e+06,1
4,B03,6.524307e-07,0.906600,0.263941,1.532730e+06,1
...,...,...,...,...,...,...
91,E04,-2.076123e-07,0.895734,0.254311,-4.816669e+06,6
92,F05,-2.118150e-07,0.658653,0.253639,-4.721102e+06,6
93,F04,-2.181697e-07,0.585321,0.255100,-4.583589e+06,6
94,A12,-2.788953e-07,0.920436,0.327502,-3.585574e+06,6


In [34]:
df_mut_stats = pd.DataFrame()
for plate_name, df_p in df_mut.groupby("Plate"):
    plate = int(plate_name[-1])
    sheet_data = plate_data[plate_data.Plate == plate].drop(columns="Plate")  #.set_index("Well")
    
    df_p["Well"] = df_p.Well.apply(lambda w: f"{w[0]}{int(w[1:]):02d}")
    # df_p = df_p.set_index("Well")

    df_p_stats = pd.merge(df_p, sheet_data, left_on="Well", right_on="Well")
    print(len(df_p), len(df_p_stats))

    df_mut_stats = pd.concat([df_mut_stats, df_p_stats])
df_mut_stats

52 52
43 43


,barcode_plate,name,refseq,variant,index,Plate,Well,Barcode,ID,P value,...,Average mutation frequency,Alignment Count,Average error rate,P adj. value,AA_seq,AA_mutations,slope,r2,y-intercept,1/slope
0,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,5.0,GsAdh_2025-04-16_plate5,A06,RB01_NB06,GsAdh_2025-04-16_plate5_A6,0.000000e+00,...,0.988095,84.0,0.000000,NaN,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,P50S_G84D_K216N,4.148571e-07,0.981295,0.260448,2.410469e+06
1,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,8.0,GsAdh_2025-04-16_plate5,A09,RB01_NB09,GsAdh_2025-04-16_plate5_A9,5.481139e-141,...,0.971429,70.0,0.014286,5.261893e-139,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,F147L,1.883223e-07,0.289903,0.284857,5.310047e+06
2,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,9.0,GsAdh_2025-04-16_plate5,A10,RB01_NB10,GsAdh_2025-04-16_plate5_A10,6.028883e-34,...,0.916667,12.0,0.083333,5.787728e-32,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,V70A_I251T,-7.927568e-08,0.113331,0.300822,-1.261421e+07
3,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,10.0,GsAdh_2025-04-16_plate5,A11,RB01_NB11,GsAdh_2025-04-16_plate5_A11,9.406419e-13,...,1.000000,6.0,0.000000,9.030162e-11,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,E314G,-9.524406e-09,0.003182,0.296995,-1.049934e+08
4,1,GsAdh_2025-04-16_plate5,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,11.0,GsAdh_2025-04-16_plate5,A12,RB01_NB12,GsAdh_2025-04-16_plate5_A12,3.808144e-07,...,0.800000,5.0,0.200000,3.655818e-05,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,,-7.497115e-09,0.012695,0.297973,-1.333846e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,65.0,GsAdh_2025-04-16_plate6,F06,RB02_NB66,GsAdh_2025-04-16_plate6_F6,1.543818e-27,...,0.933333,15.0,0.066667,1.482065e-25,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,Y171N,-6.613615e-09,0.003724,0.249420,-1.512032e+08
39,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,66.0,GsAdh_2025-04-16_plate6,F07,RB02_NB67,GsAdh_2025-04-16_plate6_F7,9.598843e-75,...,0.956522,23.0,0.000000,9.214889e-73,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,A180T,2.346722e-07,0.808475,0.247111,4.261263e+06
40,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,67.0,GsAdh_2025-04-16_plate6,F08,RB02_NB68,GsAdh_2025-04-16_plate6_F8,8.925972e-59,...,1.000000,27.0,0.000000,8.568933e-57,MKAAVVEQFKKPLQVKEVEKPKISYGEVLVRIKACGVCHTDLHAAH...,Y171N,1.347975e-07,0.392745,0.262289,7.418535e+06
41,2,GsAdh_2025-04-16_plate6,ATGAAAGCTGCAGTTGTGGAACAATTTAAAAAGCCGTTACAAGTGA...,NaN,68.0,GsAdh_2025-04-16_plate6,F09,RB02_NB69,GsAdh_2025-04-16_plate6_F9,0.000000e+00,...,0.927928,37.0,0.036036,NaN,MKAAVVEQFKKPLQVKEVEKPKISYGEV*,L29*,1.955329e-07,0.615141,0.254234,5.114229e+06


In [35]:
df_mut_stats.to_csv("250507/250507_variants_aa_mut_seq_stats.csv", index=False)

In [18]:
df_mut.Plate.value_counts()

Plate
GsAdh_2025-04-16_plate5    52
GsAdh_2025-04-16_plate6    43
Name: count, dtype: int64

In [19]:
df_mut.Well.value_counts()

Well
A6     2
F2     2
D8     2
D11    2
E4     2
      ..
D10    1
D3     1
D2     1
C6     1
E9     1
Name: count, Length: 62, dtype: int64